In [1]:
from lxml.html import parse
from urllib import request
import csv
import lxml
from lxml.html.clean import Cleaner

def unpack(row, kind='td'):
    data = row.findall('.//%s' % kind)
    return [val.text_content() for val in data]

URLs = {'Australia':'https://en.wikipedia.org/wiki/2015_Australian_Grand_Prix',
       'Malaysia':'https://en.wikipedia.org/wiki/2015_Malaysian_Grand_Prix',
       'China':'https://en.wikipedia.org/wiki/2015_Chinese_Grand_Prix',
       'Bahrain':'https://en.wikipedia.org/wiki/2015_Bahrain_Grand_Prix',
       'Spain':'https://en.wikipedia.org/wiki/2015_Spanish_Grand_Prix',
       'Monaco':'https://en.wikipedia.org/wiki/2015_Monaco_Grand_Prix',
       'Canada':'https://en.wikipedia.org/wiki/2015_Canadian_Grand_Prix',
       'Austria':'https://en.wikipedia.org/wiki/2015_Austrian_Grand_Prix',
       'England':'https://en.wikipedia.org/wiki/2015_British_Grand_Prix',
       'Hungary':'https://en.wikipedia.org/wiki/2015_Hungarian_Grand_Prix',
       'Belgium':'https://en.wikipedia.org/wiki/2015_Belgian_Grand_Prix',
       'Italy':'https://en.wikipedia.org/wiki/2015_Italian_Grand_Prix',
       'Singapore':'https://en.wikipedia.org/wiki/2015_Singapore_Grand_Prix',
       'Japan':'https://en.wikipedia.org/wiki/2015_Japanese_Grand_Prix',
       'Russia':'https://en.wikipedia.org/wiki/2015_Russian_Grand_Prix',
       'America':'https://en.wikipedia.org/wiki/2015_United_States_Grand_Prix',
       'Mexico':'https://en.wikipedia.org/wiki/2015_Mexican_Grand_Prix',
       'Brazil':'https://en.wikipedia.org/wiki/2015_Brazilian_Grand_Prix',
       'Abu Dhabi':'https://en.wikipedia.org/wiki/2015_Abu_Dhabi_Grand_Prix',
       }

# URLs = {'Abu Dhabi':'https://en.wikipedia.org/wiki/2015_Abu_Dhabi_Grand_Prix'}

def download(race):
    URL = URLs[race]
    parsed = parse(request.urlopen(URL))
    doc = parsed.getroot()
    #Remove superscript tags and their content
    cleaner = Cleaner(kill_tags=['sup'])
    doc = cleaner.clean_html(doc)
    tables = doc.findall('.//table')
    #QUALIFYING
    qualifying_results = tables[1]
    rows = qualifying_results.findall('.//tr')
    #Pull headers and data, then merge them together
    header = [unpack(row, kind='th') for row in rows]
    data = [unpack(row, kind='td') for row in rows]
    for i in range(len(data)):
        for c in range(len(header[i])-1, -1, -1):
            data[i].insert(0, header[i][c])
    for i in range(len(data)):
        for c in range(len(data[i])):
            data[i][c] = data[i][c].strip().replace(',', '')
    #Save Qualfiying results as .csv
    with open('results/' + race + '_Qualifying.csv', 'wt', newline='') as f:
        file = csv.writer(f)
        file.writerows(data)
    #RACE
    race_results = tables[2]
    rows = race_results.findall('.//tr')
    #Pull headers and data, then merge them together
    header = [unpack(row, kind='th') for row in rows]
    data = [unpack(row, kind='td') for row in rows]
    for i in range(len(data)):
        for c in range(len(header[i])-1, -1, -1):
            data[i].insert(0, header[i][c].strip())
    for i in range(len(data)):
        for c in range(len(data[i])):
            data[i][c] = data[i][c].strip().replace(',', '').replace('\r\n', '')
    #Total laps and fastest lap are stored in a metatable, appended to race results
    metatable = tables[0]
    metarows = metatable.findall('.//tr')
    metaheaders = [unpack(row, kind='th') for row in metarows]
    metadata = [unpack(row, kind='td') for row in metarows]
    #Save Race results as .csv
    with open('results/' + race + '_Race.csv', 'wt', newline='') as f:
        file = csv.writer(f)
        file.writerows(data)
        file.writerows([['Total Laps', int(metadata[metaheaders.index(['Distance'])][0].split()[0])]])
        file.writerows([['Fastest Lap', metadata[metaheaders.index(['Fastest lap']) + 1][0].strip()]])

In [2]:
for URL in URLs.keys():
    download(URL)